In [1]:
import numpy as np
import pandas as pd
from glob import glob
import xml.etree.ElementTree as xet
import cv2

In [2]:
import os

In [3]:
# Directorio actual
BASE_DIR = os.getcwd()

In [4]:
df = pd.read_csv('labels.csv')
df['filepath'] = df['filepath'].apply(lambda x: x.replace('\\','/'))
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images/162454-PLACA.xml,87,279,266,341
1,./images/N107.xml,207,356,174,287
2,./images/N113.xml,39,108,129,157
3,./images/N62.xml,289,421,188,232
4,./images/162489-PLACA.xml,1,444,115,250


In [5]:
for i in df.index:
    temp = df['filepath'][i].split("./")[1]
    temp = os.path.join(BASE_DIR, temp)
    temp = temp.replace('\\','/')
    df['filepath'][i] = temp

<ipython-input-5-d321d392f0fe>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['filepath'][i] = temp


In [6]:
#df['filepath'][0]
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/Users/user/Desktop/extracciontextoplaca/curso...,87,279,266,341
1,/Users/user/Desktop/extracciontextoplaca/curso...,207,356,174,287
2,/Users/user/Desktop/extracciontextoplaca/curso...,39,108,129,157
3,/Users/user/Desktop/extracciontextoplaca/curso...,289,421,188,232
4,/Users/user/Desktop/extracciontextoplaca/curso...,1,444,115,250


In [7]:
df['filepath'][0]

'/Users/user/Desktop/extracciontextoplaca/cursoUdemyNumberplateRecognition/Project_Files/4_modulosrecomendadoprofesor_YOLO/images/162454-PLACA.xml'

### Etiquetas yolo
    center_x, center_y, width , height

In [8]:
# parseo
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    
    filename = f'images/{name}'
    filepath = os.path.join(BASE_DIR,filename)
    filepath = filepath.replace('\\','/')

    # acho y alto
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filepath, width, height


df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)

In [9]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,/Users/user/Desktop/extracciontextoplaca/curso...,87,279,266,341,/Users/user/Desktop/extracciontextoplaca/curso...,375,500
1,/Users/user/Desktop/extracciontextoplaca/curso...,207,356,174,287,/Users/user/Desktop/extracciontextoplaca/curso...,441,363
2,/Users/user/Desktop/extracciontextoplaca/curso...,39,108,129,157,/Users/user/Desktop/extracciontextoplaca/curso...,400,300
3,/Users/user/Desktop/extracciontextoplaca/curso...,289,421,188,232,/Users/user/Desktop/extracciontextoplaca/curso...,500,269
4,/Users/user/Desktop/extracciontextoplaca/curso...,1,444,115,250,/Users/user/Desktop/extracciontextoplaca/curso...,500,375


In [10]:
# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [11]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,/Users/user/Desktop/extracciontextoplaca/curso...,87,279,266,341,/Users/user/Desktop/extracciontextoplaca/curso...,375,500,0.488000,0.607000,0.512000,0.150000
1,/Users/user/Desktop/extracciontextoplaca/curso...,207,356,174,287,/Users/user/Desktop/extracciontextoplaca/curso...,441,363,0.638322,0.634986,0.337868,0.311295
2,/Users/user/Desktop/extracciontextoplaca/curso...,39,108,129,157,/Users/user/Desktop/extracciontextoplaca/curso...,400,300,0.183750,0.476667,0.172500,0.093333
3,/Users/user/Desktop/extracciontextoplaca/curso...,289,421,188,232,/Users/user/Desktop/extracciontextoplaca/curso...,500,269,0.710000,0.780669,0.264000,0.163569
4,/Users/user/Desktop/extracciontextoplaca/curso...,1,444,115,250,/Users/user/Desktop/extracciontextoplaca/curso...,500,375,0.445000,0.486667,0.886000,0.360000


In [14]:
df.shape
### dividir los datos de las imagenes en prueba y entrenamiento y generar 
df_train = df.iloc[:210]
df_test = df.iloc[210:]

text.file

    class_id, center_x, center_y, bb_width, bb_height

In [15]:
from shutil import copy

In [16]:
train_folder = './data_images/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copiar cada imagen dentro del folder
    copy(fname,dst_image_path)

    # generar archivos .txt los cuales tienen la informacion del label
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()


In [17]:
test_folder = './data_images/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copiar imagenes dentro del folder
    copy(fname,dst_image_path)

    # generar .txt el cual tienen la informacion del label
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()


---